Untuk menampilkan log prediksi foto yang sudah dicek, kita bisa menggunakan beberapa cara tergantung pada bagaimana Anda ingin menyimpan dan menampilkan log tersebut. Berikut langkah-langkahnya, dengan asumsi Anda menggunakan Django untuk membuat aplikasi **PhotoUploader**:

---

### **1. Menyimpan Log Prediksi ke Database**
Langkah ini melibatkan pembuatan model untuk menyimpan log prediksi di database.

#### **a. Tambahkan Model Log**
Tambahkan model di `photo_uploader/models.py` untuk menyimpan informasi log:

```python
from django.db import models

class Photo(models.Model):
    title = models.CharField(max_length=100)
    image = models.ImageField(upload_to='photos/')
    uploaded_at = models.DateTimeField(auto_now_add=True)

class PredictionLog(models.Model):
    photo = models.ForeignKey(Photo, on_delete=models.CASCADE)  # Hubungkan dengan model Photo
    prediction = models.CharField(max_length=255)               # Hasil prediksi
    created_at = models.DateTimeField(auto_now_add=True)         # Waktu prediksi
```

#### **b. Migrasikan Model**
Jalankan perintah berikut untuk membuat tabel baru di database:

```bash
python manage.py makemigrations
python manage.py migrate
```

#### **c. Simpan Log di Views**
Saat prediksi dilakukan, simpan log prediksi ke tabel `PredictionLog`. Misalnya, di `photo_uploader/views.py`:

```python
from django.shortcuts import render
from .models import Photo, PredictionLog
from tensorflow.keras.models import load_model
import os

# Load model saat aplikasi dimulai
model_path = os.path.join(settings.BASE_DIR, 'models', 'model.h5')
model = load_model(model_path)

def predict_photo(request, photo_id):
    photo = Photo.objects.get(id=photo_id)
    # Lakukan prediksi (gunakan model sesuai implementasi Anda)
    image_path = photo.image.path
    prediction = model.predict(process_image(image_path))  # Sesuaikan fungsi `process_image`

    # Simpan log prediksi
    PredictionLog.objects.create(photo=photo, prediction=str(prediction))

    # Kirim hasil prediksi ke template
    return render(request, 'photo_uploader/predict.html', {'photo': photo, 'prediction': prediction})
```

---

### **2. Menampilkan Log di Halaman Web**
#### **a. Buat View untuk Menampilkan Log**
Tambahkan view untuk menampilkan semua log prediksi di `photo_uploader/views.py`:

```python
def prediction_logs(request):
    logs = PredictionLog.objects.select_related('photo').order_by('-created_at')
    return render(request, 'photo_uploader/logs.html', {'logs': logs})
```

#### **b. Buat Template untuk Log**
Buat file `photo_uploader/templates/photo_uploader/logs.html`:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Prediction Logs</title>
</head>
<body>
    <h1>Prediction Logs</h1>
    <table border="1">
        <thead>
            <tr>
                <th>Photo</th>
                <th>Prediction</th>
                <th>Timestamp</th>
            </tr>
        </thead>
        <tbody>
            {% for log in logs %}
                <tr>
                    <td>{{ log.photo.title }}</td>
                    <td>{{ log.prediction }}</td>
                    <td>{{ log.created_at }}</td>
                </tr>
            {% endfor %}
        </tbody>
    </table>
</body>
</html>
```

---

### **3. Tambahkan URL untuk Log**
Di `photo_uploader/urls.py`, tambahkan URL baru:

```python
from django.urls import path
from . import views

urlpatterns = [
    path('logs/', views.prediction_logs, name='prediction_logs'),
]
```

Akses log melalui URL `http://127.0.0.1:8000/logs/`.

---

### **4. Alternatif: Simpan Log di File**
Jika Anda ingin menyimpan log ke file daripada database, gunakan Python logging:

#### **a. Konfigurasikan Logging**
Di `settings.py`, tambahkan konfigurasi logging:

```python
LOGGING = {
    'version': 1,
    'handlers': {
        'file': {
            'level': 'INFO',
            'class': 'logging.FileHandler',
            'filename': os.path.join(BASE_DIR, 'logs', 'predictions.log'),
        },
    },
    'loggers': {
        'django': {
            'handlers': ['file'],
            'level': 'INFO',
            'propagate': True,
        },
    },
}
```

#### **b. Simpan Log di Views**
Di `photo_uploader/views.py`, tambahkan logging:

```python
import logging

logger = logging.getLogger('django')

def predict_photo(request, photo_id):
    photo = Photo.objects.get(id=photo_id)
    prediction = model.predict(process_image(photo.image.path))
    
    # Simpan log ke file
    logger.info(f"Photo: {photo.title}, Prediction: {prediction}")

    return render(request, 'photo_uploader/predict.html', {'photo': photo, 'prediction': prediction})
```

---

### **5. Menampilkan Log File di Halaman Web**
Jika log disimpan di file, Anda bisa membaca file log dan menampilkannya di halaman web:

```python
def prediction_logs(request):
    log_path = os.path.join(settings.BASE_DIR, 'logs', 'predictions.log')
    with open(log_path, 'r') as log_file:
        log_lines = log_file.readlines()
    return render(request, 'photo_uploader/logs.html', {'logs': log_lines})
```

Di template, tampilkan log seperti ini:

```html
<pre>
    {% for log in logs %}
        {{ log }}
    {% endfor %}
</pre>
```

---

### **6. Pilihan Terbaik**
- Gunakan **database** jika log akan sering diakses atau dianalisis.
- Gunakan **file** untuk logging sederhana atau pengujian awal.

Pilih sesuai kebutuhan proyek Anda.